*  Saurab Faruque
* <mark>saf130@pitt.edu</mark>

##Setup

In [99]:
#mount Drive to work with collab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [100]:
#importing pandas and numpy libraries
import pandas as pd
import numpy as np
import datetime


In [101]:
#read .csv file through Pandas as a dataframe
movielens = pd.read_csv('/content/drive/My Drive/intropy/Session_3/ml1mdata2000.csv',
                       na_values= '?')

In [102]:
movielens.head()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
0,1.0,1193.0,5,978300760,F,1.0,10.0,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,2.0,1193.0,5,978298413,M,56.0,16.0,70072,One Flew Over the Cuckoo's Nest (1975),Drama
2,12.0,1193.0,4,978220179,M,25.0,12.0,32793,One Flew Over the Cuckoo's Nest (1975),Drama
3,15.0,1193.0,4,978199279,M,25.0,7.0,22903,One Flew Over the Cuckoo's Nest (1975),Drama
4,17.0,1193.0,5,978158471,M,50.0,1.0,95350,One Flew Over the Cuckoo's Nest (1975),Drama


In [103]:
movielens.tail()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
1000204,5949.0,2198.0,5,958846401,M,18.0,17.0,47901,Modulations (1998),Documentary
1000205,5675.0,2703.0,3,976029116,M,35.0,14.0,30030,Broken Vessels (1998),Drama
1000206,5780.0,2845.0,1,958153068,M,18.0,17.0,92886,White Boys (1999),Drama
1000207,5851.0,3607.0,5,957756608,F,18.0,20.0,55410,One Little Indian (1973),Comedy|Drama|Western
1000208,5938.0,2909.0,4,957273353,M,25.0,1.0,35401,"Five Wives, Three Secretaries and Me (1998)",Documentary


##1. How many movies were rated in the fourth quarter of 2000?

In [104]:
#converting timestamp to date
movielens['date'] = pd.to_datetime(movielens['timestamp'], unit = 's', errors='coerce')

In [105]:
#separating date components into separate columns
movielens['year'] = pd.DatetimeIndex(movielens['date']).year
movielens['month'] = pd.DatetimeIndex(movielens['date']).month
movielens.tail()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres,date,year,month
1000204,5949.0,2198.0,5,958846401,M,18.0,17.0,47901,Modulations (1998),Documentary,2000-05-20 18:13:21,2000,5
1000205,5675.0,2703.0,3,976029116,M,35.0,14.0,30030,Broken Vessels (1998),Drama,2000-12-05 15:11:56,2000,12
1000206,5780.0,2845.0,1,958153068,M,18.0,17.0,92886,White Boys (1999),Drama,2000-05-12 17:37:48,2000,5
1000207,5851.0,3607.0,5,957756608,F,18.0,20.0,55410,One Little Indian (1973),Comedy|Drama|Western,2000-05-08 03:30:08,2000,5
1000208,5938.0,2909.0,4,957273353,M,25.0,1.0,35401,"Five Wives, Three Secretaries and Me (1998)",Documentary,2000-05-02 13:15:53,2000,5


In [106]:
#categorizing items into quarters based on the month (Q1 = Jan-March, Q2 = April - June, Q3 = July - Sept, Q4 = Oct - Dec)
movielens['quarter']=pd.cut(movielens['month'], bins=[1,3,6,9,12], include_lowest=True, labels=['Q1', 'Q2', 'Q3', 'Q4'])
movielens.tail()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres,date,year,month,quarter
1000204,5949.0,2198.0,5,958846401,M,18.0,17.0,47901,Modulations (1998),Documentary,2000-05-20 18:13:21,2000,5,Q2
1000205,5675.0,2703.0,3,976029116,M,35.0,14.0,30030,Broken Vessels (1998),Drama,2000-12-05 15:11:56,2000,12,Q4
1000206,5780.0,2845.0,1,958153068,M,18.0,17.0,92886,White Boys (1999),Drama,2000-05-12 17:37:48,2000,5,Q2
1000207,5851.0,3607.0,5,957756608,F,18.0,20.0,55410,One Little Indian (1973),Comedy|Drama|Western,2000-05-08 03:30:08,2000,5,Q2
1000208,5938.0,2909.0,4,957273353,M,25.0,1.0,35401,"Five Wives, Three Secretaries and Me (1998)",Documentary,2000-05-02 13:15:53,2000,5,Q2


In [107]:
#making a dataframe that counts the number of records for each movie based on movie_id

movielensgroup_bymovie=movielens.groupby(['movie_id'])['movie_id'].agg('count').reset_index(name='totalrecords')
movielensgroup_bymovie

#I recognize there are missing movie_ids for movies that do in fact have a title based on what is contained in the dataset
 #(i.e., we can know what the NaN movie_id truly is based on other rows that have the same title, but it's just not filled with the movie_id for the row where we have NaN for movie_id).
 #I address this at the end of this assignment.

,movie_id,totalrecords
0,1.0,2077
1,2.0,701
2,3.0,478
3,4.0,170
4,5.0,296
...,...,...
3700,3948.0,862
3701,3949.0,304
3702,3950.0,54
3703,3951.0,40


In [108]:
#only the movies that were rated in Q4
q4movies = movielens.loc[movielens['quarter']=='Q4']
#only the movies that were rated in Q4 of the year 2000
q4movies2000 = q4movies.loc[q4movies['year']==2000]
q4movies2000

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres,date,year,month,quarter
0,1.0,1193.0,5,978300760,F,1.0,10.0,48067,One Flew Over the Cuckoo's Nest (1975),Drama,2000-12-31 22:12:40,2000,12,Q4
1,2.0,1193.0,5,978298413,M,56.0,16.0,70072,One Flew Over the Cuckoo's Nest (1975),Drama,2000-12-31 21:33:33,2000,12,Q4
2,12.0,1193.0,4,978220179,M,25.0,12.0,32793,One Flew Over the Cuckoo's Nest (1975),Drama,2000-12-30 23:49:39,2000,12,Q4
3,15.0,1193.0,4,978199279,M,25.0,7.0,22903,One Flew Over the Cuckoo's Nest (1975),Drama,2000-12-30 18:01:19,2000,12,Q4
4,17.0,1193.0,5,978158471,M,50.0,1.0,95350,One Flew Over the Cuckoo's Nest (1975),Drama,2000-12-30 06:41:11,2000,12,Q4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000139,4169.0,701.0,4,976588990,M,50.0,0.0,66048,Daens (1992),Drama,2000-12-12 02:43:10,2000,12,Q4
1000149,4169.0,138.0,4,975804273,M,50.0,0.0,66048,"Neon Bible, The (1995)",Drama,2000-12-03 00:44:33,2000,12,Q4
1000151,4169.0,530.0,4,976589311,M,50.0,0.0,66048,Second Best (1994),Drama,2000-12-12 02:48:31,2000,12,Q4
1000153,4169.0,1070.0,3,971580375,M,50.0,0.0,66048,Macao (1952),Adventure,2000-10-15 03:26:15,2000,10,Q4


In [109]:
#grouping by movie_id so each unique movie is only counted once
q4movies2000group_bymovie=q4movies2000.groupby(['movie_id'])['movie_id'].agg('count').reset_index(name='counts')
q4movies2000_count = q4movies2000group_bymovie.shape[0]

Answer to question 1:

In [110]:
print(f'Number of movies rated in quarter 4 of the year 2000: {q4movies2000_count}')

Number of movies rated in quarter 4 of the year 2000: 3596


##2. Create a dataframe that compiles the count of missing values in the Movie Lens dataset.
Your dataframe should have the following columns: (a) Movie ID, (b) total records associated with the movie, (c) count of records with any missing values, and (d) the proportion of records for the movie with any missing values.

In [111]:
# number records with any missing values
movielens.loc[movielens.isna().any(axis=1)].shape[0]

414

In [112]:
movielens.loc[movielens.isna().any(axis='columns')].index

Int64Index([      8,       9,      10,      11,      12,      13,      14,
                 15,      16,      17,
            ...
             999964,  999965, 1000159, 1000160, 1000161, 1000162, 1000163,
            1000164, 1000165, 1000166],
           dtype='int64', length=414)

In [113]:
# records with any missing values
movielens.loc[movielens.isna().any(axis=1)]

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres,date,year,month,quarter
8,28.0,1193.0,3,978125194,F,25.0,1.0,14607,One Flew Over the Cuckoo's Nest (1975),NaN,2000-12-29 21:26:34,2000,12,Q4
9,33.0,1193.0,5,978557765,M,45.0,3.0,55421,One Flew Over the Cuckoo's Nest (1975),NaN,2001-01-03 21:36:05,2001,1,Q1
10,39.0,1193.0,5,978043535,M,18.0,4.0,61820,One Flew Over the Cuckoo's Nest (1975),NaN,2000-12-28 22:45:35,2000,12,Q4
11,42.0,1193.0,3,978038981,M,25.0,8.0,24502,One Flew Over the Cuckoo's Nest (1975),NaN,2000-12-28 21:29:41,2000,12,Q4
12,44.0,1193.0,4,978018995,M,45.0,17.0,98052,One Flew Over the Cuckoo's Nest (1975),NaN,2000-12-28 15:56:35,2000,12,Q4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000162,4486.0,133.0,1,965013057,M,35.0,8.0,98674,NaN,Comedy|Drama,2000-07-31 03:10:57,2000,7,Q3
1000163,4553.0,3290.0,3,964551353,F,35.0,16.0,48237,NaN,Comedy,2000-07-25 18:55:53,2000,7,Q3
1000164,4553.0,1709.0,2,964551353,F,35.0,16.0,48237,NaN,Thriller,2000-07-25 18:55:53,2000,7,Q3
1000165,4572.0,3164.0,4,964460301,F,1.0,10.0,17036,NaN,Drama,2000-07-24 17:38:21,2000,7,Q3


In [114]:
#making dummy variable for if row has at least 1 missing value
movielens['missing'] = 0 #dummy
movielens.loc[movielens.isna().any(axis=1), 'missing']=1 #mark row if at least one missing value

In [115]:
#display rows that have at least 1 missing value
movielens.loc[movielens['missing']==1]


,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres,date,year,month,quarter,missing
8,28.0,1193.0,3,978125194,F,25.0,1.0,14607,One Flew Over the Cuckoo's Nest (1975),NaN,2000-12-29 21:26:34,2000,12,Q4,1
9,33.0,1193.0,5,978557765,M,45.0,3.0,55421,One Flew Over the Cuckoo's Nest (1975),NaN,2001-01-03 21:36:05,2001,1,Q1,1
10,39.0,1193.0,5,978043535,M,18.0,4.0,61820,One Flew Over the Cuckoo's Nest (1975),NaN,2000-12-28 22:45:35,2000,12,Q4,1
11,42.0,1193.0,3,978038981,M,25.0,8.0,24502,One Flew Over the Cuckoo's Nest (1975),NaN,2000-12-28 21:29:41,2000,12,Q4,1
12,44.0,1193.0,4,978018995,M,45.0,17.0,98052,One Flew Over the Cuckoo's Nest (1975),NaN,2000-12-28 15:56:35,2000,12,Q4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000162,4486.0,133.0,1,965013057,M,35.0,8.0,98674,NaN,Comedy|Drama,2000-07-31 03:10:57,2000,7,Q3,1
1000163,4553.0,3290.0,3,964551353,F,35.0,16.0,48237,NaN,Comedy,2000-07-25 18:55:53,2000,7,Q3,1
1000164,4553.0,1709.0,2,964551353,F,35.0,16.0,48237,NaN,Thriller,2000-07-25 18:55:53,2000,7,Q3,1
1000165,4572.0,3164.0,4,964460301,F,1.0,10.0,17036,NaN,Drama,2000-07-24 17:38:21,2000,7,Q3,1


In [116]:
#count the number of records for each movie
movielensgroup_bymovie=movielens.groupby(['movie_id'])['movie_id'].agg('count').reset_index(name='totalrecords')
movielensgroup_bymovie

,movie_id,totalrecords
0,1.0,2077
1,2.0,701
2,3.0,478
3,4.0,170
4,5.0,296
...,...,...
3700,3948.0,862
3701,3949.0,304
3702,3950.0,54
3703,3951.0,40


In [117]:
#count number of missing records for each movie
movielensmiss=movielens.groupby(['movie_id'])['missing'].agg('sum').reset_index(name='missingcount')
movielensmiss

,movie_id,missingcount
0,1.0,0
1,2.0,0
2,3.0,0
3,4.0,0
4,5.0,0
...,...,...
3700,3948.0,0
3701,3949.0,0
3702,3950.0,0
3703,3951.0,0


In [118]:
#count the number of records for each movie
movielensgroup=pd.merge(movielensgroup_bymovie, movielensmiss, how='outer')
movielensgroup

,movie_id,totalrecords,missingcount
0,1.0,2077,0
1,2.0,701,0
2,3.0,478,0
3,4.0,170,0
4,5.0,296,0
...,...,...,...
3700,3948.0,862,0
3701,3949.0,304,0
3702,3950.0,54,0
3703,3951.0,40,0


2) Create a dataframe that compiles the count of missing values in the Movie Lens dataset. Your dataframe should have the following columns:

a) Movie ID

b) total records associated with the movie

c) count of records with any missing values

d) the proportion of records for the movie with any missing values

In [119]:
#add column that shows proportion of missing records for a given movie as a percent
movielensgroup['percent_missing']=(movielensgroup.missingcount/movielensgroup.totalrecords)*100
movielensgroup

,movie_id,totalrecords,missingcount,percent_missing
0,1.0,2077,0,0.0
1,2.0,701,0,0.0
2,3.0,478,0,0.0
3,4.0,170,0,0.0
4,5.0,296,0,0.0
...,...,...,...,...
3700,3948.0,862,0,0.0
3701,3949.0,304,0,0.0
3702,3950.0,54,0,0.0
3703,3951.0,40,0,0.0


##3. Create a "rating category"
that is coded based of the value of "movie rating": 1 & 2 are coded as "Okay"; 3 & 4 are coded as "Good"; 5 is coded as "Excellent".

In [120]:
#create rating categories
movielens['rating_category']=pd.cut(movielens['rating'], bins=[1,2,4,5], include_lowest=True, labels=['okay', 'good', 'excellent'])
movielens.head()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres,date,year,month,quarter,missing,rating_category
0,1.0,1193.0,5,978300760,F,1.0,10.0,48067,One Flew Over the Cuckoo's Nest (1975),Drama,2000-12-31 22:12:40,2000,12,Q4,0,excellent
1,2.0,1193.0,5,978298413,M,56.0,16.0,70072,One Flew Over the Cuckoo's Nest (1975),Drama,2000-12-31 21:33:33,2000,12,Q4,0,excellent
2,12.0,1193.0,4,978220179,M,25.0,12.0,32793,One Flew Over the Cuckoo's Nest (1975),Drama,2000-12-30 23:49:39,2000,12,Q4,0,good
3,15.0,1193.0,4,978199279,M,25.0,7.0,22903,One Flew Over the Cuckoo's Nest (1975),Drama,2000-12-30 18:01:19,2000,12,Q4,0,good
4,17.0,1193.0,5,978158471,M,50.0,1.0,95350,One Flew Over the Cuckoo's Nest (1975),Drama,2000-12-30 06:41:11,2000,12,Q4,0,excellent


##4. Create a dataframe with the following columns:
(a) Movie ID, (b) Average rating of the movie, (c) absolute rank of the movie based on its rating, and (d) percentile rank of the movie based on its rating.


In [121]:
#avg rating for each movie
movierating_avg=movielens[['movie_id','rating']].groupby('movie_id').mean().reset_index()
movierating_avg.rename(columns={'rating': 'avgrating'}, inplace=True)

movierating_avg

,movie_id,avgrating
0,1.0,4.146846
1,2.0,3.201141
2,3.0,3.016736
3,4.0,2.729412
4,5.0,3.006757
...,...,...
3700,3948.0,3.635731
3701,3949.0,4.115132
3702,3950.0,3.666667
3703,3951.0,3.900000


In [122]:
#adding column indicating absolute rank for each movie based on its avg rating
movierating_avg['absrank_avgrating']=movierating_avg['avgrating'].rank(ascending=False)
movierating_avg

,movie_id,avgrating,absrank_avgrating
0,1.0,4.146846,203.0
1,2.0,3.201141,2117.0
2,3.0,3.016736,2440.0
3,4.0,2.729412,2914.0
4,5.0,3.006757,2453.0
...,...,...,...
3700,3948.0,3.635731,1168.0
3701,3949.0,4.115132,231.0
3702,3950.0,3.666667,1099.5
3703,3951.0,3.900000,592.0


Create a dataframe with the following columns:

(a) Movie ID

(b) Average rating of the movie

(c) absolute rank of the movie based on its rating

(d) percentile rank of the movie based on its rating

In [123]:
#adding column indicating percentile rank for each movie based on its avg rating
movierating_avg['pctrank_avgrating']=movierating_avg['avgrating'].rank(pct=True)
movierating_avg

,movie_id,avgrating,absrank_avgrating,pctrank_avgrating
0,1.0,4.146846,203.0,0.945479
1,2.0,3.201141,2117.0,0.428880
2,3.0,3.016736,2440.0,0.341700
3,4.0,2.729412,2914.0,0.213765
4,5.0,3.006757,2453.0,0.338192
...,...,...,...,...
3700,3948.0,3.635731,1168.0,0.685020
3701,3949.0,4.115132,231.0,0.937922
3702,3950.0,3.666667,1099.5,0.703509
3703,3951.0,3.900000,592.0,0.840486


##5. Create a table (pivot or crosstab)
that shows the average ratings of movies by Zip code. Then, query the table for "James Bond Zip codes", that is, Zip codes that contain the term "007" anywhere in them.

In [124]:
#avg ratings of movies by zip code
movielenstable=pd.pivot_table(movielens, values='rating', index=['zip', 'movie_id'], aggfunc=np.mean)
movielenstable.rename(columns={"rating":"avg_rating"}, inplace=True)
movielenstable

avg_rating
zip        movie_id            
01702-7224 253.0            3.0
           480.0            3.0
           606.0            4.0
           724.0            3.0
           742.0            5.0
...                         ...
99945      3147.0           5.0
           3578.0           5.0
           3745.0           4.0
           3793.0           5.0
           3948.0           4.0

[872922 rows x 1 columns]

In [125]:
#query for james bond zipcodes
JamesBond_Zipcodes = movielenstable.reset_index().loc[movielenstable.reset_index().zip.str.contains('007')]
JamesBond_Zipcodes

,zip,movie_id,avg_rating
104742,20007,17.0,5.0
104743,20007,25.0,5.0
104744,20007,32.0,3.0
104745,20007,34.0,3.0
104746,20007,36.0,5.0
...,...,...,...
839909,98007,3114.0,5.0
839910,98007,3260.0,3.0
839911,98007,3307.0,4.0
839912,98007,3654.0,3.0


In [126]:
#remaking pivot table but with movie titles instead of movie_id
#avg ratings of movies by zip code
movielenstable_title=pd.pivot_table(movielens, values='rating', index=['zip', 'title'], aggfunc=np.mean)
movielenstable_title.rename(columns={"rating":"avg_rating"}, inplace=True)
movielenstable_title

avg_rating
zip        title                                                   
01702-7224 Abbott and Costello Meet Frankenstein (1948)         3.0
           Alien (1979)                                         3.0
           Beach Party (1963)                                   3.0
           Bikini Beach (1964)                                  3.0
           Birds, The (1963)                                    4.0
...                                                             ...
99945      Toy Story (1995)                                     4.0
           Toy Story 2 (1999)                                   5.0
           Tron (1982)                                          4.0
           Twin Dragons (Shuang long hui) (1992)                5.0
           X-Men (2000)                                         5.0

[872850 rows x 1 columns]

In [127]:
#query for james bond zipcodes
JamesBond_Zipcodes_titles = movielenstable_title.reset_index().loc[movielenstable_title.reset_index().zip.str.contains('007')]
JamesBond_Zipcodes_titles

,zip,title,avg_rating
104736,20007,28 Days (2000),3.0
104737,20007,Alien (1979),4.0
104738,20007,All the King's Men (1949),3.0
104739,20007,Almost Famous (2000),5.0
104740,20007,American Beauty (1999),4.5
...,...,...,...
839844,98007,Shakespeare in Love (1998),5.0
839845,98007,Star Wars: Episode VI - Return of the Jedi (1983),3.0
839846,98007,There's Something About Mary (1998),3.0
839847,98007,Toy Story (1995),5.0


#The next section is if I had completed the assignment by filling NaNs where possible given the data present in the dataset.

#####First part of the code is the same as the assignment above

In [128]:
#importing pandas and numpy libraries
import pandas as pd
import numpy as np
import datetime


In [129]:
#read .csv file through Pandas as a dataframe
movielens = pd.read_csv('/content/drive/My Drive/intropy/Session_3/ml1mdata2000.csv',
                       na_values= '?')

In [130]:
movielens.head()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
0,1.0,1193.0,5,978300760,F,1.0,10.0,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,2.0,1193.0,5,978298413,M,56.0,16.0,70072,One Flew Over the Cuckoo's Nest (1975),Drama
2,12.0,1193.0,4,978220179,M,25.0,12.0,32793,One Flew Over the Cuckoo's Nest (1975),Drama
3,15.0,1193.0,4,978199279,M,25.0,7.0,22903,One Flew Over the Cuckoo's Nest (1975),Drama
4,17.0,1193.0,5,978158471,M,50.0,1.0,95350,One Flew Over the Cuckoo's Nest (1975),Drama


In [131]:
movielens.tail()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
1000204,5949.0,2198.0,5,958846401,M,18.0,17.0,47901,Modulations (1998),Documentary
1000205,5675.0,2703.0,3,976029116,M,35.0,14.0,30030,Broken Vessels (1998),Drama
1000206,5780.0,2845.0,1,958153068,M,18.0,17.0,92886,White Boys (1999),Drama
1000207,5851.0,3607.0,5,957756608,F,18.0,20.0,55410,One Little Indian (1973),Comedy|Drama|Western
1000208,5938.0,2909.0,4,957273353,M,25.0,1.0,35401,"Five Wives, Three Secretaries and Me (1998)",Documentary


1. How many movies were rated in the fourth quarter of 2000?

In [132]:
#converting timestamp to date
movielens['date'] = pd.to_datetime(movielens['timestamp'], unit = 's', errors='coerce')

In [133]:
#separating date components into separate columns
movielens['year'] = pd.DatetimeIndex(movielens['date']).year
movielens['month'] = pd.DatetimeIndex(movielens['date']).month
movielens.tail()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres,date,year,month
1000204,5949.0,2198.0,5,958846401,M,18.0,17.0,47901,Modulations (1998),Documentary,2000-05-20 18:13:21,2000,5
1000205,5675.0,2703.0,3,976029116,M,35.0,14.0,30030,Broken Vessels (1998),Drama,2000-12-05 15:11:56,2000,12
1000206,5780.0,2845.0,1,958153068,M,18.0,17.0,92886,White Boys (1999),Drama,2000-05-12 17:37:48,2000,5
1000207,5851.0,3607.0,5,957756608,F,18.0,20.0,55410,One Little Indian (1973),Comedy|Drama|Western,2000-05-08 03:30:08,2000,5
1000208,5938.0,2909.0,4,957273353,M,25.0,1.0,35401,"Five Wives, Three Secretaries and Me (1998)",Documentary,2000-05-02 13:15:53,2000,5


In [134]:
#categorizing items into quarters based on the month (Q1 = Jan-March, Q2 = April - June, Q3 = July - Sept, Q4 = Oct - Dec)
movielens['quarter']=pd.cut(movielens['month'], bins=[1,3,6,9,12], include_lowest=True, labels=['Q1', 'Q2', 'Q3', 'Q4'])
movielens.tail()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres,date,year,month,quarter
1000204,5949.0,2198.0,5,958846401,M,18.0,17.0,47901,Modulations (1998),Documentary,2000-05-20 18:13:21,2000,5,Q2
1000205,5675.0,2703.0,3,976029116,M,35.0,14.0,30030,Broken Vessels (1998),Drama,2000-12-05 15:11:56,2000,12,Q4
1000206,5780.0,2845.0,1,958153068,M,18.0,17.0,92886,White Boys (1999),Drama,2000-05-12 17:37:48,2000,5,Q2
1000207,5851.0,3607.0,5,957756608,F,18.0,20.0,55410,One Little Indian (1973),Comedy|Drama|Western,2000-05-08 03:30:08,2000,5,Q2
1000208,5938.0,2909.0,4,957273353,M,25.0,1.0,35401,"Five Wives, Three Secretaries and Me (1998)",Documentary,2000-05-02 13:15:53,2000,5,Q2


#####This next section is different from my original completion of the assignment above to take into account and filling missing values for movie_id

In [135]:
# rows where 'movie_id' is NaN
missing_movie_id = movielens[movielens['movie_id'].isna()].shape[0]
print("Rows with missing movie_id:")
print(missing_movie_id)

# rows where 'title' is NaN
missing_title = movielens[movielens['title'].isna()].shape[0]
print("Rows with missing title:")
print(missing_title)

Rows with missing movie_id:
80
Rows with missing title:
173


In [136]:
#  rows both 'movie_id' and 'title' are NaN
both_missing = movielens[(movielens['movie_id'].isna()) & (movielens['title'].isna())]

# Count the number of such rows
count_both_missing = both_missing.shape[0]

print(f"Number of rows where both 'movie_id' and 'title' are missing: {count_both_missing}")


Number of rows where both 'movie_id' and 'title' are missing: 0


In [137]:
#filling in NaN in movie_id and title with appropriate values based on the rest of dataset

#dictionary to map movie_id to title
id_to_title = movielens.dropna(subset=['movie_id', 'title']).set_index('movie_id')['title'].to_dict()

#dictionary to map title to movie_id
title_to_id = movielens.dropna(subset=['movie_id', 'title']).set_index('title')['movie_id'].to_dict()

# Fill missing titles based on movie_id
movielens['title'] = movielens['title'].fillna(movielens['movie_id'].map(id_to_title))

# Fill missing movie_ids based on title
movielens['movie_id'] = movielens['movie_id'].fillna(movielens['title'].map(title_to_id))

In [138]:
# Identify rows where 'movie_id' is NaN
missing_movie_id = movielens[movielens['movie_id'].isna()].shape[0]
print("Rows with missing movie_id:")
print(missing_movie_id)

Rows with missing movie_id:
1


In [139]:
#after filling in the missing movie_id or title based on the movie_id:title correspondences present in the data
#there is movie that has a missing movie_id

In [140]:
#give that movie that has a title but missing movie_ID a new movie_ID
# maximum  movie_id
max_movie_id = movielens['movie_id'].max()

# index(es) of the rows with missing movie_id
missing_movie_id_index = movielens[movielens['movie_id'].isna()].index

# Assign new unique movie_id(s) to the row(s) with missing movie_id
movielens.loc[missing_movie_id_index, 'movie_id'] = max_movie_id + 1

# Confirm that there are no more missing

missing_movie_id = movielens[movielens['movie_id'].isna()].shape[0]
print("Rows with missing movie_id:")
print(missing_movie_id)


Rows with missing movie_id:
0


#####The rest of the code is the same as above section, outputs in the section about missing variables differs

In [141]:
#making a dataframe that counts the number of records for each movie based on movie_id

movielensgroup_bymovie=movielens.groupby(['movie_id'])['movie_id'].agg('count').reset_index(name='totalrecords')
movielensgroup_bymovie

#I recognize there are missing movie_ids for movies that do in fact have a title that we can identify based on what is contained in the dataset
 #(i.e., we can know what the NaN movie_id truly is based on other rows that have the same title, but it's just not filled with the movie_id for the row where we have NaN for movie_id).
 #I address this at the end of this assignment.

,movie_id,totalrecords
0,1.0,2077
1,2.0,701
2,3.0,478
3,4.0,170
4,5.0,296
...,...,...
3701,3949.0,304
3702,3950.0,54
3703,3951.0,40
3704,3952.0,388


In [142]:
#displays only the movies that were rated in Q4
q4movies = movielens.loc[movielens['quarter']=='Q4']
#displays only the movies that were rated in Q4 of the year 2000
q4movies2000 = q4movies.loc[q4movies['year']==2000]
q4movies2000

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres,date,year,month,quarter
0,1.0,1193.0,5,978300760,F,1.0,10.0,48067,One Flew Over the Cuckoo's Nest (1975),Drama,2000-12-31 22:12:40,2000,12,Q4
1,2.0,1193.0,5,978298413,M,56.0,16.0,70072,One Flew Over the Cuckoo's Nest (1975),Drama,2000-12-31 21:33:33,2000,12,Q4
2,12.0,1193.0,4,978220179,M,25.0,12.0,32793,One Flew Over the Cuckoo's Nest (1975),Drama,2000-12-30 23:49:39,2000,12,Q4
3,15.0,1193.0,4,978199279,M,25.0,7.0,22903,One Flew Over the Cuckoo's Nest (1975),Drama,2000-12-30 18:01:19,2000,12,Q4
4,17.0,1193.0,5,978158471,M,50.0,1.0,95350,One Flew Over the Cuckoo's Nest (1975),Drama,2000-12-30 06:41:11,2000,12,Q4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000139,4169.0,701.0,4,976588990,M,50.0,0.0,66048,Daens (1992),Drama,2000-12-12 02:43:10,2000,12,Q4
1000149,4169.0,138.0,4,975804273,M,50.0,0.0,66048,"Neon Bible, The (1995)",Drama,2000-12-03 00:44:33,2000,12,Q4
1000151,4169.0,530.0,4,976589311,M,50.0,0.0,66048,Second Best (1994),Drama,2000-12-12 02:48:31,2000,12,Q4
1000153,4169.0,1070.0,3,971580375,M,50.0,0.0,66048,Macao (1952),Adventure,2000-10-15 03:26:15,2000,10,Q4


In [143]:
#grouping by movie_id so each unique movie is only counted once
q4movies2000group_bymovie=q4movies2000.groupby(['movie_id'])['movie_id'].agg('count').reset_index(name='counts')
q4movies2000_count = q4movies2000group_bymovie.shape[0]

Answer to question 1:

In [144]:
print(f'Number of movies rated in quarter 4 of the year 2000: {q4movies2000_count}')

Number of movies rated in quarter 4 of the year 2000: 3598


2. Create a dataframe that compiles the count of missing values in the Movie Lens dataset. Your dataframe should have the following columns: (a) Movie ID, (b) total records associated with the movie, (c) count of records with any missing values, and (d) the proportion of records for the movie with any missing values.

In [145]:
# number records with any missing values
movielens.loc[movielens.isna().any(axis=1)].shape[0]

253

In [146]:
movielens.loc[movielens.isna().any(axis='columns')].index

Int64Index([      8,       9,      10,      11,      12,      13,      14,
                 15,      16,      17,
            ...
             886622,  886623, 1000159, 1000160, 1000161, 1000162, 1000163,
            1000164, 1000165, 1000166],
           dtype='int64', length=253)

In [147]:
# records with any missing values
movielens.loc[movielens.isna().any(axis=1)]

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres,date,year,month,quarter
8,28.0,1193.0,3,978125194,F,25.0,1.0,14607,One Flew Over the Cuckoo's Nest (1975),NaN,2000-12-29 21:26:34,2000,12,Q4
9,33.0,1193.0,5,978557765,M,45.0,3.0,55421,One Flew Over the Cuckoo's Nest (1975),NaN,2001-01-03 21:36:05,2001,1,Q1
10,39.0,1193.0,5,978043535,M,18.0,4.0,61820,One Flew Over the Cuckoo's Nest (1975),NaN,2000-12-28 22:45:35,2000,12,Q4
11,42.0,1193.0,3,978038981,M,25.0,8.0,24502,One Flew Over the Cuckoo's Nest (1975),NaN,2000-12-28 21:29:41,2000,12,Q4
12,44.0,1193.0,4,978018995,M,45.0,17.0,98052,One Flew Over the Cuckoo's Nest (1975),NaN,2000-12-28 15:56:35,2000,12,Q4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000162,4486.0,133.0,1,965013057,M,35.0,8.0,98674,NaN,Comedy|Drama,2000-07-31 03:10:57,2000,7,Q3
1000163,4553.0,3290.0,3,964551353,F,35.0,16.0,48237,NaN,Comedy,2000-07-25 18:55:53,2000,7,Q3
1000164,4553.0,1709.0,2,964551353,F,35.0,16.0,48237,NaN,Thriller,2000-07-25 18:55:53,2000,7,Q3
1000165,4572.0,3164.0,4,964460301,F,1.0,10.0,17036,NaN,Drama,2000-07-24 17:38:21,2000,7,Q3


In [148]:
#making dummy variable for if row has at least 1 missing value
movielens['missing'] = 0 #dummy
movielens.loc[movielens.isna().any(axis=1), 'missing']=1 #mark row if at least one missing value

In [149]:
#display rows that have at least 1 missing value
movielens.loc[movielens['missing']==1]


,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres,date,year,month,quarter,missing
8,28.0,1193.0,3,978125194,F,25.0,1.0,14607,One Flew Over the Cuckoo's Nest (1975),NaN,2000-12-29 21:26:34,2000,12,Q4,1
9,33.0,1193.0,5,978557765,M,45.0,3.0,55421,One Flew Over the Cuckoo's Nest (1975),NaN,2001-01-03 21:36:05,2001,1,Q1,1
10,39.0,1193.0,5,978043535,M,18.0,4.0,61820,One Flew Over the Cuckoo's Nest (1975),NaN,2000-12-28 22:45:35,2000,12,Q4,1
11,42.0,1193.0,3,978038981,M,25.0,8.0,24502,One Flew Over the Cuckoo's Nest (1975),NaN,2000-12-28 21:29:41,2000,12,Q4,1
12,44.0,1193.0,4,978018995,M,45.0,17.0,98052,One Flew Over the Cuckoo's Nest (1975),NaN,2000-12-28 15:56:35,2000,12,Q4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000162,4486.0,133.0,1,965013057,M,35.0,8.0,98674,NaN,Comedy|Drama,2000-07-31 03:10:57,2000,7,Q3,1
1000163,4553.0,3290.0,3,964551353,F,35.0,16.0,48237,NaN,Comedy,2000-07-25 18:55:53,2000,7,Q3,1
1000164,4553.0,1709.0,2,964551353,F,35.0,16.0,48237,NaN,Thriller,2000-07-25 18:55:53,2000,7,Q3,1
1000165,4572.0,3164.0,4,964460301,F,1.0,10.0,17036,NaN,Drama,2000-07-24 17:38:21,2000,7,Q3,1


In [150]:
#count the number of records for each movie
movielensgroup_bymovie=movielens.groupby(['movie_id'])['movie_id'].agg('count').reset_index(name='totalrecords')
movielensgroup_bymovie

,movie_id,totalrecords
0,1.0,2077
1,2.0,701
2,3.0,478
3,4.0,170
4,5.0,296
...,...,...
3701,3949.0,304
3702,3950.0,54
3703,3951.0,40
3704,3952.0,388


In [151]:
#count number of missing records for each movie
movielensmiss=movielens.groupby(['movie_id'])['missing'].agg('sum').reset_index(name='missingcount')
movielensmiss

,movie_id,missingcount
0,1.0,0
1,2.0,0
2,3.0,0
3,4.0,0
4,5.0,0
...,...,...
3701,3949.0,0
3702,3950.0,0
3703,3951.0,0
3704,3952.0,0


In [152]:
#count the number of records for each movie
movielensgroup=pd.merge(movielensgroup_bymovie, movielensmiss, how='outer')
movielensgroup

,movie_id,totalrecords,missingcount
0,1.0,2077,0
1,2.0,701,0
2,3.0,478,0
3,4.0,170,0
4,5.0,296,0
...,...,...,...
3701,3949.0,304,0
3702,3950.0,54,0
3703,3951.0,40,0
3704,3952.0,388,0


2) Create a dataframe that compiles the count of missing values in the Movie Lens dataset. Your dataframe should have the following columns:

a) Movie ID

b) total records associated with the movie

c) count of records with any missing values

d) the proportion of records for the movie with any missing values

In [153]:
#add column that shows proportion of missing records for a given movie as a percent
movielensgroup['percent_missing']=(movielensgroup.missingcount/movielensgroup.totalrecords)*100
movielensgroup

,movie_id,totalrecords,missingcount,percent_missing
0,1.0,2077,0,0.0
1,2.0,701,0,0.0
2,3.0,478,0,0.0
3,4.0,170,0,0.0
4,5.0,296,0,0.0
...,...,...,...,...
3701,3949.0,304,0,0.0
3702,3950.0,54,0,0.0
3703,3951.0,40,0,0.0
3704,3952.0,388,0,0.0


3. Create a "rating category" that is coded based of the value of "movie rating": 1 & 2 are coded as "Okay"; 3 & 4 are coded as "Good"; 5 is coded as "Excellent".

In [154]:
#create rating categories
movielens['rating_category']=pd.cut(movielens['rating'], bins=[1,2,4,5], include_lowest=True, labels=['okay', 'good', 'excellent'])
movielens.head()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres,date,year,month,quarter,missing,rating_category
0,1.0,1193.0,5,978300760,F,1.0,10.0,48067,One Flew Over the Cuckoo's Nest (1975),Drama,2000-12-31 22:12:40,2000,12,Q4,0,excellent
1,2.0,1193.0,5,978298413,M,56.0,16.0,70072,One Flew Over the Cuckoo's Nest (1975),Drama,2000-12-31 21:33:33,2000,12,Q4,0,excellent
2,12.0,1193.0,4,978220179,M,25.0,12.0,32793,One Flew Over the Cuckoo's Nest (1975),Drama,2000-12-30 23:49:39,2000,12,Q4,0,good
3,15.0,1193.0,4,978199279,M,25.0,7.0,22903,One Flew Over the Cuckoo's Nest (1975),Drama,2000-12-30 18:01:19,2000,12,Q4,0,good
4,17.0,1193.0,5,978158471,M,50.0,1.0,95350,One Flew Over the Cuckoo's Nest (1975),Drama,2000-12-30 06:41:11,2000,12,Q4,0,excellent


4. Create a dataframe with the following columns: (a) Movie ID, (b) Average rating of the movie, (c) absolute rank of the movie based on its rating, and (d) percentile rank of the movie based on its rating.


In [155]:
#avg rating for each movie
movierating_avg=movielens[['movie_id','rating']].groupby('movie_id').mean().reset_index()
movierating_avg.rename(columns={'rating': 'avgrating'}, inplace=True)

movierating_avg

,movie_id,avgrating
0,1.0,4.146846
1,2.0,3.201141
2,3.0,3.016736
3,4.0,2.729412
4,5.0,3.006757
...,...,...
3701,3949.0,4.115132
3702,3950.0,3.666667
3703,3951.0,3.900000
3704,3952.0,3.780928


In [156]:
#adding column indicating absolute rank for each movie based on its avg rating
movierating_avg['absrank_avgrating']=movierating_avg['avgrating'].rank(ascending=False)
movierating_avg

,movie_id,avgrating,absrank_avgrating
0,1.0,4.146846,203.0
1,2.0,3.201141,2116.0
2,3.0,3.016736,2439.0
3,4.0,2.729412,2913.0
4,5.0,3.006757,2452.0
...,...,...,...
3701,3949.0,4.115132,231.0
3702,3950.0,3.666667,1099.5
3703,3951.0,3.900000,592.0
3704,3952.0,3.780928,831.0


Create a dataframe with the following columns:

(a) Movie ID

(b) Average rating of the movie

(c) absolute rank of the movie based on its rating

(d) percentile rank of the movie based on its rating

In [157]:
#adding column indicating percentile rank for each movie based on its avg rating
movierating_avg['pctrank_avgrating']=movierating_avg['avgrating'].rank(pct=True)
movierating_avg

,movie_id,avgrating,absrank_avgrating,pctrank_avgrating
0,1.0,4.146846,203.0,0.945494
1,2.0,3.201141,2116.0,0.429304
2,3.0,3.016736,2439.0,0.342148
3,4.0,2.729412,2913.0,0.214247
4,5.0,3.006757,2452.0,0.338640
...,...,...,...,...
3701,3949.0,4.115132,231.0,0.937938
3702,3950.0,3.666667,1099.5,0.703589
3703,3951.0,3.900000,592.0,0.840529
3704,3952.0,3.780928,831.0,0.776039


5. Create a table (pivot or crosstab) that shows the average ratings of movies by Zip code. Then, query the table for "James Bond Zip codes", that is, Zip codes that contain the term "007" anywhere in them.

In [158]:
#avg ratings of movies by zip code
movielenstable=pd.pivot_table(movielens, values='rating', index=['zip', 'movie_id'], aggfunc=np.mean)
movielenstable.rename(columns={"rating":"avg_rating"}, inplace=True)
movielenstable

avg_rating
zip        movie_id            
01702-7224 253.0            3.0
           480.0            3.0
           606.0            4.0
           724.0            3.0
           742.0            5.0
...                         ...
99945      3147.0           5.0
           3578.0           5.0
           3745.0           4.0
           3793.0           5.0
           3948.0           4.0

[872992 rows x 1 columns]

In [159]:
#query for james bond zipcodes
JamesBond_Zipcodes = movielenstable.reset_index().loc[movielenstable.reset_index().zip.str.contains('007')]
JamesBond_Zipcodes

,zip,movie_id,avg_rating
104748,20007,17.0,5.0
104749,20007,25.0,5.0
104750,20007,32.0,3.0
104751,20007,34.0,3.0
104752,20007,36.0,5.0
...,...,...,...
839979,98007,3114.0,5.0
839980,98007,3260.0,3.0
839981,98007,3307.0,4.0
839982,98007,3654.0,3.0


In [160]:
#remaking pivot table but with movie titles instead of movie_id
#avg ratings of movies by zip code
movielenstable_title=pd.pivot_table(movielens, values='rating', index=['zip', 'title'], aggfunc=np.mean)
movielenstable_title.rename(columns={"rating":"avg_rating"}, inplace=True)
movielenstable_title

avg_rating
zip        title                                                   
01702-7224 Abbott and Costello Meet Frankenstein (1948)         3.0
           Alien (1979)                                         3.0
           Beach Party (1963)                                   3.0
           Bikini Beach (1964)                                  3.0
           Birds, The (1963)                                    4.0
...                                                             ...
99945      Toy Story (1995)                                     4.0
           Toy Story 2 (1999)                                   5.0
           Tron (1982)                                          4.0
           Twin Dragons (Shuang long hui) (1992)                5.0
           X-Men (2000)                                         5.0

[872984 rows x 1 columns]

In [161]:
#query for james bond zipcodes
JamesBond_Zipcodes_titles = movielenstable_title.reset_index().loc[movielenstable_title.reset_index().zip.str.contains('007')]
JamesBond_Zipcodes_titles

,zip,title,avg_rating
104746,20007,28 Days (2000),3.0
104747,20007,Alien (1979),4.0
104748,20007,All the King's Men (1949),3.0
104749,20007,Almost Famous (2000),5.0
104750,20007,American Beauty (1999),4.5
...,...,...,...
839973,98007,Shakespeare in Love (1998),5.0
839974,98007,Star Wars: Episode VI - Return of the Jedi (1983),3.0
839975,98007,There's Something About Mary (1998),3.0
839976,98007,Toy Story (1995),5.0
